In [26]:
import pandas as pd
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
from imblearn.under_sampling import RandomUnderSampler
import numpy as np

# set random seed for reproducibility
np.random.seed(1337)
# Load your data
# Replace 'your_data_file.csv' with your actual data file
data = pd.read_csv('combined.csv')
# drop rows where length of sequence is less than 90
data = data[data['len'] >= 90]

# Tokenize the sequences at the character level
tokenizer = Tokenizer(char_level=True)
tokenizer.fit_on_texts(data['sequence'])
sequences_numeric = tokenizer.texts_to_sequences(data['sequence'])

# Pad sequences to a maximum length of 200
max_length = 300
sequences_padded = pad_sequences(sequences_numeric, maxlen=max_length, padding='post', truncating='post')

# One-hot encode the sequences
n_classes = len(tokenizer.word_index) + 1  # +1 because index 0 is not used
one_hot_sequences = np.zeros((len(sequences_padded), max_length, n_classes))
for i, seq in enumerate(sequences_padded):
    for j, amino_acid_index in enumerate(seq):
        if amino_acid_index != 0:
            one_hot_sequences[i, j, amino_acid_index] = 1

# Balancing the dataset
rus = RandomUnderSampler(random_state=42)
X_res, y_res = rus.fit_resample(one_hot_sequences.reshape(len(one_hot_sequences), -1), data['hmmer'])
X_res = X_res.reshape(-1, max_length, n_classes)

# Splitting the dataset
X_train, X_test, y_train, y_test = train_test_split(X_res, y_res, test_size=0.2, random_state=42)


In [27]:
from keras.models import Sequential
from keras.layers import LSTM, Dense, Dropout

# Model configuration
n_units = 100  # Number of units in the LSTM layer

# Build the model
model = Sequential()
model.add(LSTM(n_units, input_shape=(max_length, n_classes), return_sequences=False))
model.add(Dropout(0.2))
model.add(Dense(1, activation='sigmoid'))

model.summary()


Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_3 (LSTM)               (None, 100)               48800     
                                                                 
 dropout_2 (Dropout)         (None, 100)               0         
                                                                 
 dense_3 (Dense)             (None, 1)                 101       
                                                                 
Total params: 48901 (191.02 KB)
Trainable params: 48901 (191.02 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [29]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs= 50, batch_size=64)


Epoch 1/20
75/75 [==============================] - 10s 106ms/step - loss: 0.5113 - accuracy: 0.7510 - val_loss: 0.4959 - val_accuracy: 0.7717
Epoch 2/20
75/75 [==============================] - 7s 98ms/step - loss: 0.5115 - accuracy: 0.7723 - val_loss: 0.5083 - val_accuracy: 0.7717
Epoch 3/20
75/75 [==============================] - 7s 96ms/step - loss: 0.5014 - accuracy: 0.7634 - val_loss: 0.4885 - val_accuracy: 0.7751
Epoch 4/20
75/75 [==============================] - 7s 96ms/step - loss: 0.5298 - accuracy: 0.7506 - val_loss: 0.6263 - val_accuracy: 0.7043
Epoch 5/20
75/75 [==============================] - 7s 96ms/step - loss: 0.5848 - accuracy: 0.6823 - val_loss: 0.5547 - val_accuracy: 0.7094
Epoch 6/20
75/75 [==============================] - 7s 98ms/step - loss: 0.5531 - accuracy: 0.7135 - val_loss: 0.5452 - val_accuracy: 0.7152
Epoch 7/20
75/75 [==============================] - 7s 98ms/step - loss: 0.5502 - accuracy: 0.7091 - val_loss: 0.5545 - val_accuracy: 0.7094
Epoch 8/20
